# グローバーのアルゴリズム
グローバーのアルゴリズムは未整序のデータの検索が高速にできるというアルゴリズムです。マーキングの部分にあたるオラクルと呼ばれる関数部分に検索のアルゴリズムを入れましょう。

全体の流れは下記の通りです。

１、重ね合わせ
２、探したいデータのマーキング
３、マーキングを可視化する振幅増幅反転

マーキングと振幅増幅反転を適切な回数繰り返すと精度が上がります。こちらをBlueqatで実装してみたいと思います。

## マーキング
まず2量子ビットのGroverからです。2量子ビットの組み合わせの場合の数は、00,01,10,11の4通りです。その中から特定の組み合わせを抜き出してみたいと思います。それを実現するにはゲート操作をつかって、「解に対応する状態ベクトルだけに-1がかかる対角行列を数学的に作り」ます。
 

In [0]:
'''
#00
[[-1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]]

#01
[[ 1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]]

#10
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]]

#11
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]
 '''

このように、マーキングしたい状態ベクトルに対応する部分に-1を設定した行列を作ってかけます。

## マーキング
マーキング回路は下記のように重ね合わせの後に上記のゲートをかけます。下記のようにHのあとに各々のオラクルをつけます。*はコントロールゲート、Zはターゲットゲートです。対応するblueqatコードも併記します。

In [0]:
'''
#00
--H--S--*--S--
--H--S--Z--S--
'''

Circuit(2).h[:].s[:].cz[0,1].s[:]

'''
#01
--H-----*-----
--H--S--Z--S--
'''

Circuit(2).h[:].s[1].cz[0,1].s[1]

'''
#10
--H--S--*--S--
--H-----Z-----
'''

Circuit(2).h[:].s[0].cz[0,1].s[0]

'''
#11
--H-----*-----
--H-----Z-----
'''

Circuit(2).h[:].cz[0,1]

## 振幅増幅反転
振幅増幅反転は、対角項が-1、非対角項が+1の行列を用意することでマーキングした回路にかけることで、マーキングしたものの振幅が増幅されます。振幅増幅反転のユニタリ変換は各パターン共通となっています。こちらをBlueqatに直してみます。

In [0]:
Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].m[:]

'''
--H-X-*-X-H--
--H-X-Z-X-H--
'''


## 回路の実装
では、実際の回路の実装です。

In [8]:
from blueqat import Circuit

#振幅増幅反転
a = Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].m[:]

'''
#00回路
--H--S--*--S----H-X-*-X-H--
--H--S--Z--S----H-X-Z-X-H--
'''

(Circuit(2).h[:].s[:].cz[0,1].s[:] + a).run(shots=100)

Counter({'00': 100})

In [9]:
'''
#01回路
--H-----*-------H-X-*-X-H--
--H--S--Z--S---H-X-Z-X-H--
'''

(Circuit(2).h[:].s[1].cz[0,1].s[1] + a).run(shots=100)

Counter({'01': 100})

In [10]:
'''
#10回路
--H--S--*--S----H-X-*-X-H--
--H-----Z--------H-X-Z-X-H--
'''
(Circuit(2).h[:].s[0].cz[0,1].s[0] + a).run(shots=100)

Counter({'10': 100})

In [11]:
'''
#11回路
--H-----*-------H-X-*-X-H--
--H-----Z-------H-X-Z-X-H--
'''
(Circuit(2).h[:].cz[0,1] + a).run(shots=100)

Counter({'11': 100})

このようにできました。Groverのアルゴリズムが身近になりました。

# 概要

## Oracle
まずはこのアルゴリズムに出てくるオラクルから説明します。
簡単にいうと、ある入力に対し 0 あるいは 1 を返す関数です。
例として、物を買う状況を考えましょう。お肉屋さんで買える物を 1、 買えない物を 0 とします。

すると以下の状況になります。

<img src="./img/016/016_0.png" width="40%">

これを用いるとパソコンはお肉屋さんで買えなく、牛肉は買えるので、

<img src="./img/016/016_1.png" width="20%">

となります。

このように 0 と 1 に分ける関数をオラクルと言います。

今回は 1 つだけ 1 を返す Oracle を考えます。
つまり $f$ の入力が 00...00 から 11...11 の $2^n$ 個でどこか一つだけ 1 を返し、それ以外は全て 0 を返すもとします。

<img src="./img/014/014_02_0.png" width="18%">

$x = \omega$ で 1 を返すとしています。

### Amplitude amplification (振幅増幅)
振幅という言葉が出てきました。
各状態の係数のことを**振幅**、確率のことを**確率振幅**と言います。

以下の式を見てみます。

<img src="./img/014/014_02_1.png" width="50%">

この場合、矢印の左側では確率振幅は全て $1/4$ となり、
右側の状態の確率振幅は 00 が 1 となっています。

Amplitude amplification は言葉の通りこの確率振幅を上のように増幅させるアルゴリズムです。

以下の図を用いて具体的な計算を説明します。

<img width="30%" src="https://upload.wikimedia.org/wikipedia/commons/1/16/Grovers_algorithm_geometry.png">

参考: https://en.wikipedia.org/wiki/Grover%27s_algorithm

まずは記号の説明をします。
s は全ビットの重ね合わせです。

<img src="./img/014/014_02_2.png" width="30%">

このうち、振幅増幅させたい状態を $\omega$ としています。

<img src="./img/014/014_02_3.png" width="25%">

x 番目に1が入っているとします。

s' は s から $\omega$ を除いたベクトルです。

<img src="./img/014/014_02_4.png" width="20%">

$\omega$ に垂直なことがわかります。

次に上の図のように s' と $\omega$ から成る単位円を考えます。
ただし s' と $\omega$ の長さが違うので

<img src="./img/014/014_02_5.png" width="30%">

として長さを共に 1 にします。

このことから、単位円上のベクトル $\psi$ は以下のように表せます。

<img src="./img/014/014_02_6.png" width="20%">

$U_{\omega}$ は $s'$ を軸に $\psi$ を反転させる行列です。
すなわち $\psi$ を $-\phi$ 回転させます。

<img src="./img/014/014_02_7.png" width="50%">

$U_s$ は $\psi$ を s で反転させる行列です。

<img src="./img/014/014_02_8.png" width="80%">

#### 概要
アルゴリズムの概要は以下のようになります。

1 s を $U_{\omega}$ を用いて s' で折り返す。

2 $U_{\omega}$s を $U_s$ を用いて s で折り返す。

以上の流れを詳しく説明していきます。

#### 1. s' に関する折り返し
s について上の図のように $\theta$ を用いて

<img src="./img/014/014_02_9.png" width="25%">

と定義します。

このとき

<img src="./img/014/014_02_10.png" width="30%">

と表せます。

$U_{\omega}$ で s を s' を軸に折り返します。
上の図から以下のようにかけます。

<img src="./img/014/014_02_11.png" width="60%">

この操作に関しては $\omega$ のみに作用しているので $U_{\omega}$ は上で述べた Oracle を表していることがわかります。

#### 2. s に関する折り返し
$U_s$ で $U_{\omega}$s を s を軸に折り返します。

<img src="./img/014/014_02_12.png" width="40%">

ここで $2\theta$ 回転させれば良いので

<img src="./img/014/014_02_13.png" width="35%">

具体的に $\cos$ と $\sin$ を求めると加法定理から

<img src="./img/014/014_02_14.png" width="45%">

よって、s', $\omega$ を用いると

<img src="./img/014/014_02_15.png" width="35%">

この操作によって $2^n$ 個の振幅のうち $\omega$ が他のよりも約３倍大きくなりました。
以上で振幅増幅させることができました。

## Grover's algorithm
このアルゴリズムでは上の Oracle が与えられたときに $\omega$ を見つけることができます。

内容としては振幅増幅を繰り返し行い、s を $\omega$ に近づけるということをします。

上の式から 1 回振幅増幅を行うと

<img src="./img/014/014_02_16.png" width="25%">

となりました。

よって、$n$ 回振幅増幅を行うと

<img src="./img/014/014_02_17.png" width="70%">

と表せます。

振幅増幅を行うに連れて上の図の初期状態 s が段々と $\omega$ に近づいて行くことがわかります。
つまり $\omega$ の確率振幅が 1 になっていきます。
ただこのまま繰り返ししても回転し続けるだけなので繰り返す回数を考える必要があります。

$\omega$ の確率振幅を 1 にするので

<img src="./img/014/014_02_18.png" width="50%">

以上から振幅増幅する回数が決まり、$\omega$ が取れることがわかりました。

## 補足 (量子プログラミングへの応用)
量子プログラミングでは振幅増幅は以下のように考えます。

$U_{\omega}$ はシンプルに s の定義から $s = s' + \omega$ と分けて考えると

<img src="./img/014/014_02_19.png" width="20%">

となります。
つまりZ、CZゲートなどのように特定の状態だけ符号を変えるゲートを用いれば良いことがわかります。

$U_s$ は上の図から幾何的に考えると以下のようにかけます。

<img src="./img/014/014_02_20.png" width="35%">

よって $U_s = 2|s\rangle \langle s| - I$ となります。

さらに $U_s$ は以下のように分解できます。

<img src="./img/014/014_02_21.png" width="70%">

ここで $2|0^n><0^n|-I$ に関して

<img src="./img/014/014_02_22.png" width="35%">

と表せる。

これは

<img src="./img/014/014_02_23.png" width="18%">

の性質から

<img src="./img/014/014_02_24.png" width="28%">

とかける。

以上から Grover's algorithm をゲートで書き直すことができた。